In [1]:
import gentrl
import torch
import pandas as pd
torch.cuda.set_device(0)

import warnings
warnings.simplefilter('ignore')

In [2]:
from moses.metrics import mol_passes_filters, QED, SA, logP
from moses.metrics.utils import get_n_rings, get_mol


def get_num_rings_6(mol):
    r = mol.GetRingInfo()
    return len([x for x in r.AtomRings() if len(x) > 6])


def penalized_logP(mol_or_smiles, masked=False, default=-5):
    mol = get_mol(mol_or_smiles)
    if mol is None:
        return default
    reward = logP(mol) - SA(mol) - get_num_rings_6(mol)
    if masked and not mol_passes_filters(mol):
        return default
    return reward

In [3]:
# ! wget https://media.githubusercontent.com/media/molecularsets/moses/master/data/dataset_v1.csv

In [4]:
# df = pd.read_csv('dataset_v1.csv')
# df = df[df['SPLIT'] == 'train']
# df['plogP'] = df['SMILES'].apply(penalized_logP)
# df.to_csv('train_plogp_plogpm.csv', index=None)

In [5]:
enc = gentrl.RNNEncoder(latent_size=50)
dec = gentrl.DilConvDecoder(latent_input_size=50)
model = gentrl.GENTRL(enc, dec, 50 * [('c', 20)], [('c', 20)], beta=0.001)
model.cuda();

In [6]:
md = gentrl.MolecularDataset(sources=[
    {'path':'train_plogp_plogpm.csv',
     'smiles': 'SMILES',
     'prob': 1,
     'plogP' : 'plogP',
    }], 
    props=['plogP'])

from torch.utils.data import DataLoader
train_loader = DataLoader(md, batch_size=50, shuffle=True, num_workers=1, drop_last=True)

In [ ]:
model.train_as_vaelp(train_loader, lr=1e-4)

Epoch 0 :
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 2.481;rec: -2.394;kl: -65.52;log_p_y_by_z: -1.524;log_p_z_by_y: -71.45;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 1.793;rec: -1.696;kl: -54.66;log_p_y_by_z: -1.521;log_p_z_by_y: -59.18;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 1.566;rec: -1.457;kl: -44.0;log_p_y_by_z: -1.523;log_p_z_by_y: -49.53;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 1.419;rec: -1.306;kl: -38.62;log_p_y_by_z: -1.514;log_p_z_by_y: -46.3;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 1.329;rec: -1.218;kl: -35.5;log_p_y_by_z: -1.471;log_p_z_by_y: -45.39;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 1.254;rec: -1.148;kl: -33.41;log_p_y_by_z: -1.395;log_p_z_by_y: -45.24;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 1.203;rec: -1.102;kl: -32.01;log_p_y_by_z: -1.334;log_p_z_by_y: -43.5

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.634;rec: -0.6286;kl: -15.44;log_p_y_by_z: -0.2086;log_p_z_by_y: -57.06;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.6376;rec: -0.6268;kl: -15.37;log_p_y_by_z: -0.262;log_p_z_by_y: -57.56;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.6411;rec: -0.6336;kl: -15.52;log_p_y_by_z: -0.2302;log_p_z_by_y: -57.67;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.6195;rec: -0.6204;kl: -15.6;log_p_y_by_z: -0.1469;log_p_z_by_y: -57.71;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.6336;rec: -0.6281;kl: -15.24;log_p_y_by_z: -0.2075;log_p_z_by_y: -57.75;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.6154;rec: -0.6176;kl: -14.82;log_p_y_by_z: -0.1266;log_p_z_by_y: -57.64;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.6198;rec: -0.6168;kl: -14.31;log_p_y_by_z: -0.1731;log_p_z_by_y: -57.7;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.6134;rec: -0.6106;kl: -14.63;

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.4921;rec: -0.5095;kl: -12.94;log_p_y_by_z: 0.04426;log_p_z_by_y: -66.85;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.4862;rec: -0.5077;kl: -13.1;log_p_y_by_z: 0.08311;log_p_z_by_y: -67.06;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.4872;rec: -0.507;kl: -13.61;log_p_y_by_z: 0.06217;log_p_z_by_y: -66.86;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.4893;rec: -0.5101;kl: -14.02;log_p_y_by_z: 0.06707;log_p_z_by_y: -67.51;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.4796;rec: -0.4996;kl: -13.75;log_p_y_by_z: 0.06254;log_p_z_by_y: -67.96;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.4771;rec: -0.4963;kl: -13.95;log_p_y_by_z: 0.05257;log_p_z_by_y: -67.92;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.4711;rec: -0.4957;kl: -13.61;log_p_y_by_z: 0.11;log_p_z_by_y: -68.13;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.47;rec: -0.4958;kl: -14.12;log

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.4027;rec: -0.4322;kl: -14.25;log_p_y_by_z: 0.1524;log_p_z_by_y: -75.65;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.3913;rec: -0.4179;kl: -14.09;log_p_y_by_z: 0.1255;log_p_z_by_y: -76.19;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.3903;rec: -0.4194;kl: -13.99;log_p_y_by_z: 0.1518;log_p_z_by_y: -76.13;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.4068;rec: -0.429;kl: -13.3;log_p_y_by_z: 0.08864;log_p_z_by_y: -75.93;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.3939;rec: -0.4248;kl: -13.83;log_p_y_by_z: 0.1714;log_p_z_by_y: -75.94;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.3856;rec: -0.4126;kl: -13.6;log_p_y_by_z: 0.1341;log_p_z_by_y: -76.2;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.3918;rec: -0.4259;kl: -13.55;log_p_y_by_z: 0.205;log_p_z_by_y: -76.25;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.3837;rec: -0.4152;kl: -14.33;log_p_y

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.3321;rec: -0.368;kl: -13.53;log_p_y_by_z: 0.2231;log_p_z_by_y: -82.63;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.3236;rec: -0.3627;kl: -13.1;log_p_y_by_z: 0.2595;log_p_z_by_y: -82.29;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.3251;rec: -0.3626;kl: -13.73;log_p_y_by_z: 0.2385;log_p_z_by_y: -82.84;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.3279;rec: -0.3663;kl: -14.13;log_p_y_by_z: 0.2422;log_p_z_by_y: -83.02;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.3307;rec: -0.3699;kl: -12.96;log_p_y_by_z: 0.262;log_p_z_by_y: -82.92;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.3345;rec: -0.3705;kl: -13.41;log_p_y_by_z: 0.2259;log_p_z_by_y: -83.09;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.3193;rec: -0.3601;kl: -13.78;log_p_y_by_z: 0.2696;log_p_z_by_y: -83.47;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: 0.3204;rec: -0.3596;kl: -12.67;log_p_

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



!!!!!!!!!!!!!!!!!!!!!!!loss: -0.1302;rec: -0.09902;kl: -158.2;log_p_y_by_z: 0.7108;log_p_z_by_y: -241.9;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: -0.1161;rec: -0.1098;kl: -157.4;log_p_y_by_z: 0.6847;log_p_z_by_y: -241.1;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: -0.1223;rec: -0.1052;kl: -156.9;log_p_y_by_z: 0.706;log_p_z_by_y: -241.5;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: -0.1267;rec: -0.1025;kl: -157.5;log_p_y_by_z: 0.7169;log_p_z_by_y: -241.8;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: -0.1273;rec: -0.1043;kl: -157.9;log_p_y_by_z: 0.7372;log_p_z_by_y: -242.1;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: -0.1288;rec: -0.1016;kl: -159.3;log_p_y_by_z: 0.7108;log_p_z_by_y: -242.9;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: -0.1247;rec: -0.1052;kl: -157.6;log_p_y_by_z: 0.7224;log_p_z_by_y: -242.3;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: -0.13;rec: -0.09743;kl: -157.8;log_p_y_by_z: 0.6964;log_p_z_by_y: -2

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: -0.1225;rec: -0.102;kl: -159.1;log_p_y_by_z: 0.6546;log_p_z_by_y: -243.6;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: -0.1223;rec: -0.1032;kl: -158.9;log_p_y_by_z: 0.6674;log_p_z_by_y: -243.1;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: -0.1261;rec: -0.1035;kl: -158.9;log_p_y_by_z: 0.7068;log_p_z_by_y: -243.2;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: -0.1236;rec: -0.1027;kl: -159.5;log_p_y_by_z: 0.6684;log_p_z_by_y: -243.1;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: -0.1274;rec: -0.1025;kl: -158.6;log_p_y_by_z: 0.7138;log_p_z_by_y: -242.7;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: -0.1256;rec: -0.1054;kl: -160.4;log_p_y_by_z: 0.7064;log_p_z_by_y: -243.3;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: -0.1263;rec: -0.1057;kl: -158.8;log_p_y_by_z: 0.732;log_p_z_by_y: -242.7;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: -0.1333;rec: -0.09815;kl: -15

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: -0.1332;rec: -0.101;kl: -160.2;log_p_y_by_z: 0.74;log_p_z_by_y: -244.8;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: -0.1315;rec: -0.1005;kl: -159.9;log_p_y_by_z: 0.7211;log_p_z_by_y: -244.6;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: -0.1227;rec: -0.102;kl: -160.2;log_p_y_by_z: 0.6456;log_p_z_by_y: -244.2;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: -0.1271;rec: -0.1014;kl: -160.8;log_p_y_by_z: 0.6768;log_p_z_by_y: -245.5;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: -0.1297;rec: -0.1016;kl: -159.9;log_p_y_by_z: 0.7145;log_p_z_by_y: -245.1;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: -0.1348;rec: -0.1009;kl: -160.2;log_p_y_by_z: 0.7556;log_p_z_by_y: -244.8;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: -0.1306;rec: -0.1018;kl: -159.8;log_p_y_by_z: 0.7261;log_p_z_by_y: -244.1;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: -0.1341;rec: -0.1001;kl: -160.4

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: -0.123;rec: -0.1027;kl: -162.1;log_p_y_by_z: 0.6366;log_p_z_by_y: -246.3;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: -0.1279;rec: -0.1012;kl: -161.7;log_p_y_by_z: 0.6742;log_p_z_by_y: -245.9;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: -0.1255;rec: -0.1053;kl: -160.2;log_p_y_by_z: 0.706;log_p_z_by_y: -246.2;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: -0.1338;rec: -0.09916;kl: -162.1;log_p_y_by_z: 0.7085;log_p_z_by_y: -246.3;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: -0.1329;rec: -0.1001;kl: -163.0;log_p_y_by_z: 0.7002;log_p_z_by_y: -247.0;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: -0.1314;rec: -0.1014;kl: -161.5;log_p_y_by_z: 0.7119;log_p_z_by_y: -246.4;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: -0.1274;rec: -0.1011;kl: -163.0;log_p_y_by_z: 0.6551;log_p_z_by_y: -246.8;
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!loss: -0.1289;rec: -0.1007;kl: -16

In [ ]:
! mkdir -p saved_gentrl

In [ ]:
model.save('./saved_gentrl/')